In [1]:
import sys
sys.path.insert(0,"/home/shackste/galaxy-generator/python_modules/")


from pdb import set_trace

# Classifier

In [2]:
from torch.nn import Module, Linear, MaxPool1d
from torch import max, Tensor
from torch.nn.parameter import Parameter
import torch.nn.functional as F
from torch.nn.modules.utils import _pair

from additional_layers import Reshape, PrintShape


class MaxOut(Module):
    """ Maxout Layer
    """
    def __init__(self, in_features, out_features, N_layers=2, **kwargs):
        super(MaxOut, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.dense = []
        for i, N_label in enumerate(label_group_sizes):
            layer = f"dense{i}"
            setattr(self, layer,
                Sequential(
                   Linear(self.in_features, self.out_features, **kwargs),
                )
            )
            self.dense.append( getattr(self, layer) )
        
        
    def forward(self, input):
        x = self.dense[0](input)
        for layer in self.dense[1:]:
            x = max(x, layer(input))
        return x

class MaxOut(Module):
    """ Maxout Layer
    """
    def __init__(self, in_features, out_features, N_layers=2, **kwargs):
        super(MaxOut, self).__init__()
        self.maxout = Sequential(
            Linear(in_features, N_layers*out_features, **kwargs),
            Reshape(1,N_layers*out_features),
            MaxPool1d(N_layers),
            Reshape(out_features),
        )
        
    def forward(self, input):
        x = self.maxout(input)
        return x

    
class Conv2dUntiedBias(Module):
    def __init__(self, height, width, in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias_init=0.1, weight_std=0.01):
        super(Conv2dUntiedBias, self).__init__() 
        kernel_size = _pair(kernel_size)
        stride = _pair(stride)
        padding = _pair(padding)
        dilation = _pair(dilation)

        if in_channels % groups != 0:
            raise ValueError('in_channels must be divisible by groups')
        if out_channels % groups != 0:
            raise ValueError('out_channels must be divisible by groups')
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        self.dilation = dilation
        self.groups = groups
        self.weight = Parameter(Tensor(
                out_channels, in_channels // groups, *kernel_size))
        self.bias = Parameter(Tensor(out_channels, height, width))
        self.reset_parameters(bias_init=bias_init, weight_std=weight_std)

    def reset_parameters(self, bias_init=None, weight_std=None):
        n = self.in_channels
        for k in self.kernel_size:
            n *= k
        if weight_std is None:
            stdv = n**-0.5
            self.weight.data.uniform_(-stdv, stdv)
        else:
            self.weight.data.uniform_(-weight_std, weight_std)            
        if bias_init is None:
            self.bias.data.uniform_(-stdv, stdv)
        else:
            self.bias.data[:] = bias_init
        

    def forward(self, input):
        if not self.kernel_size[0] % 2 and self.padding: ## one side padding 1 extra for even kernel size
            input = F.pad(input=input, pad=(1,0,1,0))
        output = F.conv2d(input, self.weight, None, self.stride,
                        self.padding, self.dilation, self.groups)
        # add untied bias
        output += self.bias.unsqueeze(0).repeat(input.size(0), 1, 1, 1)
        return output

    


In [3]:
import torchvision.models as models
from torch import squeeze, cat, stack
from torch.nn import Sequential, Linear, Conv2d, MaxPool2d, Softmax, Dropout, ReLU, Flatten
from torchvision.transforms import Compose, RandomHorizontalFlip, FiveCrop, Lambda
from torchvision.transforms.functional import rotate, hflip

from neuralnetwork import NeuralNetwork
from labeling import make_galaxy_labels_hierarchical, label_group_sizes, labels_dim
from parameter import parameter

from dataset import augment

resnet = models.resnet18(pretrained=False)
N_resnet = 512



class ImageClassifier(NeuralNetwork):
    def __init__(self):
        super(ImageClassifier, self).__init__()
#        self.conv = Sequential(*(list(resnet.children())[:-1]))
        self.conv = Sequential(
            Conv2dUntiedBias(41, 41, 3, 32, kernel_size=6),
            ReLU(),
            MaxPool2d(2),
            Conv2dUntiedBias(16, 16, 32, 64, kernel_size=5),
            ReLU(),
            MaxPool2d(2),
            Conv2dUntiedBias(6, 6, 64, 128, kernel_size=3),
            ReLU(),
            Conv2dUntiedBias(4, 4, 128, 128, kernel_size=3, weight_std=0.1),
            ReLU(),
            MaxPool2d(2),
            Flatten(),
        )
        self.dense = Sequential(
            Dropout(p=0.5),
            MaxOut(8192, 2048, bias=0.01),
            Dropout(p=0.5),
            MaxOut(2048, 2048, bias=0.01),
            Dropout(p=0.5),
            MaxOut(2048, 37, bias=0.1),
            ReLU()
        )
        
        self.augment = Compose([
            Lambda(lambda img: cat([img, hflip(img)], 0)),
            Lambda(lambda img: cat([img, rotate(img, 45)], 0)),
            FiveCrop(45),
            Lambda(lambda crops: cat([rotate(crop, ang) for crop, ang in zip(crops[:-1], (0, 90, 270, 180))], 0))
        ])
        self.N_augmentations = 16
        self.N_conv_outputs = 512

        
        self.set_optimizer(parameter.optimizer, lr=parameter.learning_rate, betas=parameter.betas)
        
        self.valid_probabilities = False
        

    def update_optimizer(self, betas=parameter.betas, **kwargs):
        self.set_optimizer(parameter.optimizer, betas=betas, **kwargs)
        
    def forward(self, images):
        batch_size = images.size(0)
        x = self.augment(images)
        x = self.conv(x)

        ## combine results of augmentations to a single vector 
        x = x.reshape(self.N_augmentations, batch_size, self.N_conv_outputs)
        x = x.permute(1,0,2)
        x = x.reshape(batch_size, self.N_augmentations*self.N_conv_outputs)
                
        x = squeeze(x)
        labels = self.dense(x)
        if self.valid_probabilities:
            labels = make_galaxy_labels_hierarchical(label_groups)
        return labels

image_size = 45

from torch import rand
classifier = ImageClassifier()
img = rand(2, 3, 64, 64)
classifier(img).shape



!!!!!!!!!!

galaxyzoo_data_cropped_nonnormalized.npy and training_solutions_rev1.csv must be placed in google drive under galaxy-generator/data/
the results will be placed there, too.



torch.Size([2, 37])

# Training

In [11]:
from time import time
import torch 

from loss import mse
from dataset import MakeDataLoader
from neuralnetwork import update_networks_on_loss

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

make_data_loader = MakeDataLoader()

batch_size = 64

learning_rate_schedule = {
    # iteration : learning_rate
    0 : 0.04,
    18000000 / batch_size : 0.004,
    23000000 / batch_size : 0.0004,
    
}

losses = []
losses_steps = 10  # number of batches between loss tracking
accuracy = []
accuracy_steps = 10 # number of batches after which to measure accuracy


classifier = ImageClassifier().to(device)
iteration = 0
epochs = 1

In [5]:
t0 = time()

for epoch in range(epochs):
    print(f"epoch {epoch+1}")
    data_loader_train = make_data_loader.get_data_loader_train(batch_size=batch_size)
    for images, labels in data_loader_train:
        if iteration == 10000:
            classifier.valid_probabilities = True
        elif iteration in learning_rate_schedule.keys():
            classifier.update_optimizer(lr=learning_rate_schedule[iteration])

        if not iteration % accuracy_steps:
            data_loader_test = make_data_loader.get_data_loader_test(batch_size=256)
            for images_test, labels_test in data_loader_test:
                labels_pred = classifier(images_test)
                break
            loss = mse(labels_test, labels_pred)
            print(f"test loss {loss.item():.4}, {iteration} iterations")
            accuracy.append(loss)
            
        labels_pred = classifier(images)
        loss = mse(labels, labels_pred)
        update_networks_on_loss(loss, classifier)
            
        if not iteration % losses_steps:
            t1 = time()
            print(f"training loss {loss.item():.4}, iteration {iteration+1}, {t1-t0:.3} s")
            t0 = t1

        iteration += 1


epoch 1
test loss 0.06546, 0 iterations
training loss 0.07434, iteration 10, 23.3 s
test loss 0.07281, 10 iterations
training loss 0.07074, iteration 20, 23.5 s
test loss 0.07507, 20 iterations


KeyboardInterrupt: 

In [13]:
from torch.utils.data import DataLoader
device == torch.device("cpu")

True

In [ ]:
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from torchvision.transforms import ToPILImage
for img, lab in data_loader:
    print(img.shape)
    print(img.permute(0,1,3,2).shape)
    break

In [ ]:
from torch import sum, rand, zeros

z = zeros(10,100)

loss_class(z+1,z), 25000000/64*2/3600
